In [1]:
# import kagglehub

# path = kagglehub.dataset_download("ealtman2019/ibm-transactions-for-anti-money-laundering-aml")

# print("Path to dataset files:", path)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
import sklearn.metrics
from sklearn.metrics import average_precision_score

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest

from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# from category_encoders import CountEncoder, TargetEncoder
from scipy.sparse import hstack

In [3]:
df = pd.read_csv('/Users/andrzej/python_programs/CODE/data/ibm-transactions-for-anti-money-laundering-aml/HI-Small_Trans.csv')
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2022/09/01 00:20,10,8000EBD30,10,8000EBD30,3697.34,US Dollar,3697.34,US Dollar,Reinvestment,0
1,2022/09/01 00:20,3208,8000F4580,1,8000F5340,0.01,US Dollar,0.01,US Dollar,Cheque,0
2,2022/09/01 00:00,3209,8000F4670,3209,8000F4670,14675.57,US Dollar,14675.57,US Dollar,Reinvestment,0
3,2022/09/01 00:02,12,8000F5030,12,8000F5030,2806.97,US Dollar,2806.97,US Dollar,Reinvestment,0
4,2022/09/01 00:06,10,8000F5200,10,8000F5200,36682.97,US Dollar,36682.97,US Dollar,Reinvestment,0


In [4]:
df.isnull().sum()

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64

In [5]:
df["Is Laundering"].value_counts()

Is Laundering
0    5073168
1       5177
Name: count, dtype: int64

In [6]:
df["Is Laundering"].value_counts(normalize=True)*100

Is Laundering
0    99.898057
1     0.101943
Name: proportion, dtype: float64

In [7]:
print(sorted(df['Receiving Currency'].unique()) == sorted(df['Payment Currency'].unique()))

True


In [8]:
print(df['Amount Received'].equals(['Amount Paid']))

False


In [9]:
df.dtypes

Timestamp              object
From Bank               int64
Account                object
To Bank                 int64
Account.1              object
Amount Received       float64
Receiving Currency     object
Amount Paid           float64
Payment Currency       object
Payment Format         object
Is Laundering           int64
dtype: object

In [10]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df['Timestamp'] = df['Timestamp'].apply(lambda x: x.value)
df['Timestamp'] = (df['Timestamp'] - df['Timestamp'].min()) / (df['Timestamp'].max() - df['Timestamp'].min())

In [11]:
cat_cols = df.select_dtypes(include='object').columns.to_list()
cat_cols

['Account',
 'Account.1',
 'Receiving Currency',
 'Payment Currency',
 'Payment Format']

In [12]:
df.shape

(5078345, 11)

In [13]:
le = LabelEncoder()
for col in cat_cols:
    df[col] = le.fit_transform(df[col])
df

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,0.000786,10,942,10,900,3697.340000,12,3697.340000,12,5,0
1,0.000786,3208,990,1,949,0.010000,12,0.010000,12,3,0
2,0.000000,3209,991,3209,940,14675.570000,12,14675.570000,12,5,0
3,0.000079,12,997,12,945,2806.970000,12,2806.970000,12,5,0
4,0.000236,10,999,10,947,36682.970000,12,36682.970000,12,5,0
...,...,...,...,...,...,...,...,...,...,...,...
5078340,0.565520,54219,495979,256398,419726,0.154978,1,0.154978,1,1,0
5078341,0.564656,15,495988,256398,419726,0.108128,1,0.108128,1,1,0
5078342,0.565323,154365,495980,256398,419726,0.004988,1,0.004988,1,1,0
5078343,0.565088,256398,495978,256398,419726,0.038417,1,0.038417,1,1,0


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           float64
 1   From Bank           int64  
 2   Account             int64  
 3   To Bank             int64  
 4   Account.1           int64  
 5   Amount Received     float64
 6   Receiving Currency  int64  
 7   Amount Paid         float64
 8   Payment Currency    int64  
 9   Payment Format      int64  
 10  Is Laundering       int64  
dtypes: float64(3), int64(8)
memory usage: 426.2 MB


In [15]:
scaler = MinMaxScaler()

In [16]:
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
df_scaled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5078345 entries, 0 to 5078344
Data columns (total 11 columns):
 #   Column              Dtype  
---  ------              -----  
 0   Timestamp           float64
 1   From Bank           float64
 2   Account             float64
 3   To Bank             float64
 4   Account.1           float64
 5   Amount Received     float64
 6   Receiving Currency  float64
 7   Amount Paid         float64
 8   Payment Currency    float64
 9   Payment Format      float64
 10  Is Laundering       float64
dtypes: float64(11)
memory usage: 426.2 MB


In [19]:
y = df_scaled['Is Laundering']
X = df_scaled.drop('Is Laundering', axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [34]:
clf = IsolationForest(random_state=42).fit(X_train, y_train)

In [35]:
preds = clf.predict(X_test)

In [ ]:
auprc = average_precision_score(y_test, preds)

print(f"AUPRC: {auprc}")

AUPRC: 0.0009538959270665196
